In [3]:
# Import the required libraries
import sqlite3
import pandas as pd
import pickle
from statistics import mode
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.pipeline import Pipeline
from joblib import load
from joblib import dump

In [4]:
# Connect to the database
conn = sqlite3.connect('data.db')
cursor = conn.cursor()

# Query and joining all the tables in the database and obtain the data as a DataFrame
query = """
SELECT c.client_id, c.age, c.job, c.marital, c.education, c.gender,
       cp.has_deposits, cp.loan, cp.has_insurance, cp.has_mortgage,
       ice.poutcome,
       b.date, b.balance, b.currency
FROM client AS c
LEFT JOIN client_products  AS cp ON c.client_id = cp.client_id
LEFT JOIN inv_campaign_eval  AS ice ON c.client_id = ice.client_id
LEFT JOIN balances AS b ON c.client_id = b.client_id
"""
df = pd.read_sql_query(query, conn)
df

,client_id,age,job,marital,education,gender,has_deposits,loan,has_insurance,has_mortgage,poutcome,date,balance,currency
0,249789938,38.0,services,married,secondary,M,yes,no,n,no,success,2022-06-21 16:21:15.291346,2633.671253,CZK
1,249789938,38.0,services,married,secondary,M,yes,no,n,no,success,2022-07-05 16:21:15.291346,2663.697443,CZK
2,249789938,38.0,services,married,secondary,M,yes,no,n,no,success,2022-07-19 16:21:15.291346,2605.350108,CZK
3,249789938,38.0,services,married,secondary,M,yes,no,n,no,success,2022-08-02 16:21:15.291346,2694.840314,CZK
4,249789938,38.0,services,married,secondary,M,yes,no,n,no,success,2022-08-16 16:21:15.291346,2656.018182,CZK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
290207,705801685,36.0,blue-collar,single,secondary,F,yes,no,n,yes,success,2023-04-11 16:22:46.440217,84.358039,CZK
290208,705801685,36.0,blue-collar,single,secondary,F,yes,no,n,yes,success,2023-04-25 16:22:46.440217,8.617271,CZK
290209,705801685,36.0,blue-collar,single,secondary,F,yes,no,n,yes,success,2023-05-09 16:22:46.440217,40.666162,CZK
290210,705801685,36.0,blue-collar,single,secondary,F,yes,no,n,yes,success,2023-05-23 16:22:46.440217,-13.947329,CZK


In [5]:
#Change the data types of the age and balance columns

def change_data_types(df):
    df['date'] = pd.to_datetime(df['date'])
    df['age'] = df['age'].astype(float)
    df['balance'] = df['balance'].astype(float)

    return df

df = change_data_types(df)
df

,client_id,age,job,marital,education,gender,has_deposits,loan,has_insurance,has_mortgage,poutcome,date,balance,currency
0,249789938,38.0,services,married,secondary,M,yes,no,n,no,success,2022-06-21 16:21:15.291346,2633.671253,CZK
1,249789938,38.0,services,married,secondary,M,yes,no,n,no,success,2022-07-05 16:21:15.291346,2663.697443,CZK
2,249789938,38.0,services,married,secondary,M,yes,no,n,no,success,2022-07-19 16:21:15.291346,2605.350108,CZK
3,249789938,38.0,services,married,secondary,M,yes,no,n,no,success,2022-08-02 16:21:15.291346,2694.840314,CZK
4,249789938,38.0,services,married,secondary,M,yes,no,n,no,success,2022-08-16 16:21:15.291346,2656.018182,CZK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
290207,705801685,36.0,blue-collar,single,secondary,F,yes,no,n,yes,success,2023-04-11 16:22:46.440217,84.358039,CZK
290208,705801685,36.0,blue-collar,single,secondary,F,yes,no,n,yes,success,2023-04-25 16:22:46.440217,8.617271,CZK
290209,705801685,36.0,blue-collar,single,secondary,F,yes,no,n,yes,success,2023-05-09 16:22:46.440217,40.666162,CZK
290210,705801685,36.0,blue-collar,single,secondary,F,yes,no,n,yes,success,2023-05-23 16:22:46.440217,-13.947329,CZK


In [6]:
#CONVERTING THE BALANCE TO CZK
def convert_to_czk(row, currency):
    if currency == 'USD':
        return row['balance'] * usd_to_czk
    elif currency == 'EURO':
        return row['balance'] * euro_to_czk
    else:
        return row['balance']

usd_to_czk = 23.4  # replace with the current exchange rate
euro_to_czk = 25.32  # replace with the current exchange rate

df['balance'] = df.apply(lambda row: convert_to_czk(row, row['currency']), axis=1)
df

,client_id,age,job,marital,education,gender,has_deposits,loan,has_insurance,has_mortgage,poutcome,date,balance,currency
0,249789938,38.0,services,married,secondary,M,yes,no,n,no,success,2022-06-21 16:21:15.291346,2633.671253,CZK
1,249789938,38.0,services,married,secondary,M,yes,no,n,no,success,2022-07-05 16:21:15.291346,2663.697443,CZK
2,249789938,38.0,services,married,secondary,M,yes,no,n,no,success,2022-07-19 16:21:15.291346,2605.350108,CZK
3,249789938,38.0,services,married,secondary,M,yes,no,n,no,success,2022-08-02 16:21:15.291346,2694.840314,CZK
4,249789938,38.0,services,married,secondary,M,yes,no,n,no,success,2022-08-16 16:21:15.291346,2656.018182,CZK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
290207,705801685,36.0,blue-collar,single,secondary,F,yes,no,n,yes,success,2023-04-11 16:22:46.440217,84.358039,CZK
290208,705801685,36.0,blue-collar,single,secondary,F,yes,no,n,yes,success,2023-04-25 16:22:46.440217,8.617271,CZK
290209,705801685,36.0,blue-collar,single,secondary,F,yes,no,n,yes,success,2023-05-09 16:22:46.440217,40.666162,CZK
290210,705801685,36.0,blue-collar,single,secondary,F,yes,no,n,yes,success,2023-05-23 16:22:46.440217,-13.947329,CZK


In [7]:
# Calculate the average spending for each client
def calculate_average_spending(df):
    # Sort the DataFrame
    df = df.sort_values(['client_id', 'date'])

    # Calculate the balance difference for each date
    df['balance_diff'] = df.groupby('client_id')['balance'].diff()

    # Calculate the average spending for each client
    average_spending = df.groupby('client_id')['balance_diff'].mean()

    # Map the average spending to the client_id in the original DataFrame
    df['average_spending'] = df['client_id'].map(average_spending)

    return df

df_spending = calculate_average_spending(df)
df_spending

,client_id,age,job,marital,education,gender,has_deposits,loan,has_insurance,has_mortgage,poutcome,date,balance,currency,balance_diff,average_spending
265876,100070076,41.0,management,divorced,tertiary,M,no,no,n,no,None,2022-06-21 16:22:36.643827,361.807400,CZK,NaN,-7.454485
265877,100070076,41.0,management,divorced,tertiary,M,no,no,n,no,None,2022-07-05 16:22:36.643827,113.142183,CZK,-248.665217,-7.454485
265878,100070076,41.0,management,divorced,tertiary,M,no,no,n,no,None,2022-07-19 16:22:36.643827,148.537614,CZK,35.395431,-7.454485
265879,100070076,41.0,management,divorced,tertiary,M,no,no,n,no,None,2022-08-02 16:22:36.643827,215.003236,CZK,66.465622,-7.454485
265880,100070076,41.0,management,divorced,tertiary,M,no,no,n,no,None,2022-08-16 16:22:36.643827,129.477982,CZK,-85.525254,-7.454485
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232695,1999956516,32.0,blue-collar,married,secondary,M,yes,no,n,yes,failure,2023-04-11 16:22:24.006798,207.392088,CZK,-99.430220,2.536917
232696,1999956516,32.0,blue-collar,married,secondary,M,yes,no,n,yes,failure,2023-04-25 16:22:24.006798,257.053986,CZK,49.661898,2.536917
232697,1999956516,32.0,blue-collar,married,secondary,M,yes,no,n,yes,failure,2023-05-09 16:22:24.006798,271.478324,CZK,14.424338,2.536917
232698,1999956516,32.0,blue-collar,married,secondary,M,yes,no,n,yes,failure,2023-05-23 16:22:24.006798,180.129715,CZK,-91.348609,2.536917


In [8]:
# Select the latest balance for each client
def select_latest_balance(df_spending):
    # Sort by 'client_id' and 'date'
    df_spending = df_spending.sort_values(['client_id', 'date'])
    
    # Calculate the average balance for each client
    average_balance = df_spending.groupby('client_id')['balance'].mean()

    # Map the average balance to the client_id in the original DataFrame
    df_spending['average_balance'] = df_spending['client_id'].map(average_balance)
    
    # Drop duplicates, keeping only the last entry for each 'client_id'
    df_spending = df_spending.drop_duplicates('client_id', keep='last')
    
    return df_spending

df_balance = select_latest_balance(df_spending)
df_balance

,client_id,age,job,marital,education,gender,has_deposits,loan,has_insurance,has_mortgage,poutcome,date,balance,currency,balance_diff,average_spending,average_balance
265901,100070076,41.0,management,divorced,tertiary,M,no,no,n,no,None,2023-06-06 16:22:36.643827,175.445269,CZK,29.953812,-7.454485,203.907928
175317,100141910,34.0,management,single,tertiary,F,no,no,n,yes,None,2023-06-06 16:22:04.265177,2559.041192,CZK,9.340310,0.444639,2544.965438
191645,100207570,34.0,management,single,tertiary,F,yes,no,n,no,None,2023-06-06 16:22:09.249972,94.055761,CZK,107.718168,2.436701,-15.679495
43705,100488560,43.0,admin.,single,secondary,F,yes,no,n,no,failure,2023-06-06 16:21:25.597087,382.755389,CZK,6.380241,5.109879,381.488613
262703,100605013,21.0,student,single,primary,M,yes,no,y,no,success,2023-06-06 16:22:35.410449,369.476219,CZK,-140.446928,0.039401,420.351449
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22879,1999463182,34.0,blue-collar,married,secondary,M,no,no,n,yes,None,2023-06-06 16:21:20.742835,1146.101785,CZK,-165.302083,-4.687871,1268.089349
130363,1999499145,31.0,management,single,tertiary,F,no,no,n,yes,None,2023-06-06 16:21:50.781726,30.830676,CZK,36.034384,-0.482484,-4.751058
87099,1999715381,24.0,blue-collar,married,secondary,M,yes,no,n,yes,None,2023-06-06 16:21:36.964752,792.767525,CZK,136.046944,6.663159,676.988885
89179,1999945413,39.0,None,single,tertiary,F,yes,no,n,no,None,2023-06-06 16:21:37.470159,5077.753230,CZK,3.199596,-3.010250,5047.496305


In [9]:
#fill the missing values in the 'job', 'marital', and 'education' columns with the mode
df_balance['job'] = df_balance['job'].fillna(mode(df_balance['job']))
df_balance['marital'] = df_balance['marital'].fillna(mode(df_balance['marital']))
df_balance['education'] = df_balance['education'].fillna(mode(df_balance['education']))
df_balance['age'] = df_balance['age'].fillna(df_balance['age'].mean())
df_balance

,client_id,age,job,marital,education,gender,has_deposits,loan,has_insurance,has_mortgage,poutcome,date,balance,currency,balance_diff,average_spending,average_balance
265901,100070076,41.0,management,divorced,tertiary,M,no,no,n,no,None,2023-06-06 16:22:36.643827,175.445269,CZK,29.953812,-7.454485,203.907928
175317,100141910,34.0,management,single,tertiary,F,no,no,n,yes,None,2023-06-06 16:22:04.265177,2559.041192,CZK,9.340310,0.444639,2544.965438
191645,100207570,34.0,management,single,tertiary,F,yes,no,n,no,None,2023-06-06 16:22:09.249972,94.055761,CZK,107.718168,2.436701,-15.679495
43705,100488560,43.0,admin.,single,secondary,F,yes,no,n,no,failure,2023-06-06 16:21:25.597087,382.755389,CZK,6.380241,5.109879,381.488613
262703,100605013,21.0,student,single,primary,M,yes,no,y,no,success,2023-06-06 16:22:35.410449,369.476219,CZK,-140.446928,0.039401,420.351449
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22879,1999463182,34.0,blue-collar,married,secondary,M,no,no,n,yes,None,2023-06-06 16:21:20.742835,1146.101785,CZK,-165.302083,-4.687871,1268.089349
130363,1999499145,31.0,management,single,tertiary,F,no,no,n,yes,None,2023-06-06 16:21:50.781726,30.830676,CZK,36.034384,-0.482484,-4.751058
87099,1999715381,24.0,blue-collar,married,secondary,M,yes,no,n,yes,None,2023-06-06 16:21:36.964752,792.767525,CZK,136.046944,6.663159,676.988885
89179,1999945413,39.0,management,single,tertiary,F,yes,no,n,no,None,2023-06-06 16:21:37.470159,5077.753230,CZK,3.199596,-3.010250,5047.496305


In [10]:
#drop the date and currency columns
df_balance = df_balance.drop(columns=['date', 'currency'])

#drop the 'balance_diff' column
df_balance = df_balance.drop(columns=['balance_diff'])
df_balance

,client_id,age,job,marital,education,gender,has_deposits,loan,has_insurance,has_mortgage,poutcome,balance,average_spending,average_balance
265901,100070076,41.0,management,divorced,tertiary,M,no,no,n,no,None,175.445269,-7.454485,203.907928
175317,100141910,34.0,management,single,tertiary,F,no,no,n,yes,None,2559.041192,0.444639,2544.965438
191645,100207570,34.0,management,single,tertiary,F,yes,no,n,no,None,94.055761,2.436701,-15.679495
43705,100488560,43.0,admin.,single,secondary,F,yes,no,n,no,failure,382.755389,5.109879,381.488613
262703,100605013,21.0,student,single,primary,M,yes,no,y,no,success,369.476219,0.039401,420.351449
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22879,1999463182,34.0,blue-collar,married,secondary,M,no,no,n,yes,None,1146.101785,-4.687871,1268.089349
130363,1999499145,31.0,management,single,tertiary,F,no,no,n,yes,None,30.830676,-0.482484,-4.751058
87099,1999715381,24.0,blue-collar,married,secondary,M,yes,no,n,yes,None,792.767525,6.663159,676.988885
89179,1999945413,39.0,management,single,tertiary,F,yes,no,n,no,None,5077.753230,-3.010250,5047.496305


In [11]:
#encode the categorical columns for cleaned table
df_dummy = pd.get_dummies(df_balance)
df_dummy

,client_id,age,balance,average_spending,average_balance,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,...,has_deposits_no,has_deposits_yes,loan_no,loan_yes,has_insurance_n,has_insurance_y,has_mortgage_no,has_mortgage_yes,poutcome_failure,poutcome_success
265901,100070076,41.0,175.445269,-7.454485,203.907928,False,False,False,False,True,...,True,False,True,False,True,False,True,False,False,False
175317,100141910,34.0,2559.041192,0.444639,2544.965438,False,False,False,False,True,...,True,False,True,False,True,False,False,True,False,False
191645,100207570,34.0,94.055761,2.436701,-15.679495,False,False,False,False,True,...,False,True,True,False,True,False,True,False,False,False
43705,100488560,43.0,382.755389,5.109879,381.488613,True,False,False,False,False,...,False,True,True,False,True,False,True,False,True,False
262703,100605013,21.0,369.476219,0.039401,420.351449,False,False,False,False,False,...,False,True,True,False,False,True,True,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22879,1999463182,34.0,1146.101785,-4.687871,1268.089349,False,True,False,False,False,...,True,False,True,False,True,False,False,True,False,False
130363,1999499145,31.0,30.830676,-0.482484,-4.751058,False,False,False,False,True,...,True,False,True,False,True,False,False,True,False,False
87099,1999715381,24.0,792.767525,6.663159,676.988885,False,True,False,False,False,...,False,True,True,False,True,False,False,True,False,False
89179,1999945413,39.0,5077.753230,-3.010250,5047.496305,False,False,False,False,True,...,False,True,True,False,True,False,True,False,False,False


In [12]:
#selecting the clients in the first campaign evaluation
df_outcome = df_balance[df_balance['poutcome'].isin(['failure', 'success'])]
df_outcome

,client_id,age,job,marital,education,gender,has_deposits,loan,has_insurance,has_mortgage,poutcome,balance,average_spending,average_balance
43705,100488560,43.0,admin.,single,secondary,F,yes,no,n,no,failure,382.755389,5.109879,381.488613
262703,100605013,21.0,student,single,primary,M,yes,no,y,no,success,369.476219,0.039401,420.351449
103115,102604801,41.0,management,married,secondary,F,no,no,n,yes,success,184.981881,-4.462468,270.449689
286857,104030718,27.0,student,single,tertiary,F,yes,no,n,no,success,340.821795,1.509400,312.058814
123967,106830158,39.0,management,married,primary,F,yes,no,n,yes,success,6981.709891,-0.950741,7008.932616
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257373,1996359586,40.0,unemployed,single,tertiary,M,yes,no,n,no,success,7957.013794,-3.877861,7956.276324
94327,1996474627,48.0,unemployed,married,secondary,M,yes,no,n,no,success,989.012654,2.509168,919.751520
33851,1997220693,42.0,admin.,married,primary,F,yes,no,n,yes,success,-248.224994,0.464894,-284.422803
187381,1997336391,30.0,management,married,tertiary,M,yes,no,n,yes,failure,8093.323617,1.771686,8090.524175


In [13]:
#change the 'poutcome' column to binary
df_outcome.loc[:, 'poutcome'] = df_outcome['poutcome'].map({'failure': 'no', 'success': 'yes'})
df_outcome

,client_id,age,job,marital,education,gender,has_deposits,loan,has_insurance,has_mortgage,poutcome,balance,average_spending,average_balance
43705,100488560,43.0,admin.,single,secondary,F,yes,no,n,no,no,382.755389,5.109879,381.488613
262703,100605013,21.0,student,single,primary,M,yes,no,y,no,yes,369.476219,0.039401,420.351449
103115,102604801,41.0,management,married,secondary,F,no,no,n,yes,yes,184.981881,-4.462468,270.449689
286857,104030718,27.0,student,single,tertiary,F,yes,no,n,no,yes,340.821795,1.509400,312.058814
123967,106830158,39.0,management,married,primary,F,yes,no,n,yes,yes,6981.709891,-0.950741,7008.932616
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257373,1996359586,40.0,unemployed,single,tertiary,M,yes,no,n,no,yes,7957.013794,-3.877861,7956.276324
94327,1996474627,48.0,unemployed,married,secondary,M,yes,no,n,no,yes,989.012654,2.509168,919.751520
33851,1997220693,42.0,admin.,married,primary,F,yes,no,n,yes,yes,-248.224994,0.464894,-284.422803
187381,1997336391,30.0,management,married,tertiary,M,yes,no,n,yes,no,8093.323617,1.771686,8090.524175


In [14]:
#encode the categorical columns for the outcome table
df_dummy_one = pd.get_dummies(df_outcome)
df_dummy_one

,client_id,age,balance,average_spending,average_balance,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,...,has_deposits_no,has_deposits_yes,loan_no,loan_yes,has_insurance_n,has_insurance_y,has_mortgage_no,has_mortgage_yes,poutcome_no,poutcome_yes
43705,100488560,43.0,382.755389,5.109879,381.488613,True,False,False,False,False,...,False,True,True,False,True,False,True,False,True,False
262703,100605013,21.0,369.476219,0.039401,420.351449,False,False,False,False,False,...,False,True,True,False,False,True,True,False,False,True
103115,102604801,41.0,184.981881,-4.462468,270.449689,False,False,False,False,True,...,True,False,True,False,True,False,False,True,False,True
286857,104030718,27.0,340.821795,1.509400,312.058814,False,False,False,False,False,...,False,True,True,False,True,False,True,False,False,True
123967,106830158,39.0,6981.709891,-0.950741,7008.932616,False,False,False,False,True,...,False,True,True,False,True,False,False,True,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257373,1996359586,40.0,7957.013794,-3.877861,7956.276324,False,False,False,False,False,...,False,True,True,False,True,False,True,False,False,True
94327,1996474627,48.0,989.012654,2.509168,919.751520,False,False,False,False,False,...,False,True,True,False,True,False,True,False,False,True
33851,1997220693,42.0,-248.224994,0.464894,-284.422803,True,False,False,False,False,...,False,True,True,False,True,False,False,True,False,True
187381,1997336391,30.0,8093.323617,1.771686,8090.524175,False,False,False,False,True,...,False,True,True,False,True,False,False,True,True,False


In [15]:
# Split the data into features and target variable
X = df_dummy_one[['age', 'balance', 'average_spending',
       'average_balance', 'job_admin.', 'job_blue-collar', 'job_entrepreneur',
       'job_housemaid', 'job_management', 'job_retired', 'job_self-employed',
       'job_services', 'job_student', 'job_technician', 'job_unemployed',
       'marital_divorced', 'marital_married', 'marital_single',
       'education_primary', 'education_secondary', 'education_tertiary',
       'has_deposits_no',
       'has_deposits_yes', 'loan_no', 'loan_yes', 'has_insurance_n',
       'has_insurance_y', 'has_mortgage_no', 'has_mortgage_yes']] # Features
y = df_dummy_one['poutcome_yes'] # Target variable

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=40, test_size=0.3)

In [16]:
# Define the pipeline
pipeline_RF = Pipeline([
    ('scaler', StandardScaler()),  # Use StandardScaler as transformer
    ('classifier', RandomForestClassifier(n_estimators=400, max_features='sqrt', max_depth=5, criterion='entropy'))  # Use RandomForestClassifier as estimator
])

# Fit the pipeline to the training data
pipeline_RF.fit(X_train, y_train)

# Initializing models
kf = KFold(n_splits=5, shuffle=True, random_state=40)
kfold_scores = cross_val_score(pipeline_RF, X, y, cv=kf)
print(f"Average K-Fold CV score: {round(kfold_scores.mean() * 100, 2)}%")

# Perform cross-validation
scores = cross_val_score(pipeline_RF, X, y, cv=kf)
print(f"Average cross-validation score: {round(scores.mean() * 100, 2)}%")

Average K-Fold CV score: 70.51%
Average cross-validation score: 70.73%


In [17]:
# Make predictions on the test set
y_pred = pipeline_RF.predict(X_test)

# Evaluate the model's performance
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {round(accuracy * 100, 2)}%")

# Print the classification report
report = classification_report(y_test, y_pred)
print(f"Classification Report: \n{report}")

Accuracy: 70.58%
Classification Report: 
              precision    recall  f1-score   support

       False       0.77      0.67      0.72       386
        True       0.64      0.75      0.69       304

    accuracy                           0.71       690
   macro avg       0.71      0.71      0.71       690
weighted avg       0.71      0.71      0.71       690



In [18]:
#saving the model
dump(pipeline_RF, 'pipeline_RF.joblib')
pipeline_RF = load('pipeline_RF.joblib')
with open('pipeline_RF.pkl', 'wb') as file:
    pickle.dump(pipeline_RF, file)
with open('pipeline_RF.pkl', 'rb') as file:
    your_random_forest_model = pickle.load(file)

In [19]:
#select the target for the second campaign
df_target = df_balance[~df_balance['poutcome'].isin(['failure', 'success'])]
df_target

,client_id,age,job,marital,education,gender,has_deposits,loan,has_insurance,has_mortgage,poutcome,balance,average_spending,average_balance
265901,100070076,41.0,management,divorced,tertiary,M,no,no,n,no,None,175.445269,-7.454485,203.907928
175317,100141910,34.0,management,single,tertiary,F,no,no,n,yes,None,2559.041192,0.444639,2544.965438
191645,100207570,34.0,management,single,tertiary,F,yes,no,n,no,None,94.055761,2.436701,-15.679495
55873,100646594,75.0,retired,divorced,primary,F,no,no,n,no,None,37136.144031,1.532749,37131.526158
141023,100741497,56.0,management,married,secondary,M,no,no,n,yes,None,240.737339,5.987920,228.186356
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209741,1999460711,53.0,admin.,divorced,secondary,M,no,yes,n,yes,None,259.851300,0.196719,215.601872
22879,1999463182,34.0,blue-collar,married,secondary,M,no,no,n,yes,None,1146.101785,-4.687871,1268.089349
130363,1999499145,31.0,management,single,tertiary,F,no,no,n,yes,None,30.830676,-0.482484,-4.751058
87099,1999715381,24.0,blue-collar,married,secondary,M,yes,no,n,yes,None,792.767525,6.663159,676.988885


In [20]:
# Select 'client_id' from the target data
client_ids = df_target['client_id']

# Select the rows with the filtered 'client_id's
X_new = df_dummy[df_dummy['client_id'].isin(client_ids)]
X_new

,client_id,age,balance,average_spending,average_balance,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,...,has_deposits_no,has_deposits_yes,loan_no,loan_yes,has_insurance_n,has_insurance_y,has_mortgage_no,has_mortgage_yes,poutcome_failure,poutcome_success
265901,100070076,41.0,175.445269,-7.454485,203.907928,False,False,False,False,True,...,True,False,True,False,True,False,True,False,False,False
175317,100141910,34.0,2559.041192,0.444639,2544.965438,False,False,False,False,True,...,True,False,True,False,True,False,False,True,False,False
191645,100207570,34.0,94.055761,2.436701,-15.679495,False,False,False,False,True,...,False,True,True,False,True,False,True,False,False,False
55873,100646594,75.0,37136.144031,1.532749,37131.526158,False,False,False,False,False,...,True,False,True,False,True,False,True,False,False,False
141023,100741497,56.0,240.737339,5.987920,228.186356,False,False,False,False,True,...,True,False,True,False,True,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209741,1999460711,53.0,259.851300,0.196719,215.601872,True,False,False,False,False,...,True,False,False,True,True,False,False,True,False,False
22879,1999463182,34.0,1146.101785,-4.687871,1268.089349,False,True,False,False,False,...,True,False,True,False,True,False,False,True,False,False
130363,1999499145,31.0,30.830676,-0.482484,-4.751058,False,False,False,False,True,...,True,False,True,False,True,False,False,True,False,False
87099,1999715381,24.0,792.767525,6.663159,676.988885,False,True,False,False,False,...,False,True,True,False,True,False,False,True,False,False


In [21]:
# Get the feature names from the training data
feature_names = X_train.columns
feature_names

Index(['age', 'balance', 'average_spending', 'average_balance', 'job_admin.',
       'job_blue-collar', 'job_entrepreneur', 'job_housemaid',
       'job_management', 'job_retired', 'job_self-employed', 'job_services',
       'job_student', 'job_technician', 'job_unemployed', 'marital_divorced',
       'marital_married', 'marital_single', 'education_primary',
       'education_secondary', 'education_tertiary', 'has_deposits_no',
       'has_deposits_yes', 'loan_no', 'loan_yes', 'has_insurance_n',
       'has_insurance_y', 'has_mortgage_no', 'has_mortgage_yes'],
      dtype='object')

In [22]:
# Filter the new data to include only the matching feature names
X_new_filtered = X_new[feature_names]
X_new_filtered

,age,balance,average_spending,average_balance,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,...,education_secondary,education_tertiary,has_deposits_no,has_deposits_yes,loan_no,loan_yes,has_insurance_n,has_insurance_y,has_mortgage_no,has_mortgage_yes
265901,41.0,175.445269,-7.454485,203.907928,False,False,False,False,True,False,...,False,True,True,False,True,False,True,False,True,False
175317,34.0,2559.041192,0.444639,2544.965438,False,False,False,False,True,False,...,False,True,True,False,True,False,True,False,False,True
191645,34.0,94.055761,2.436701,-15.679495,False,False,False,False,True,False,...,False,True,False,True,True,False,True,False,True,False
55873,75.0,37136.144031,1.532749,37131.526158,False,False,False,False,False,True,...,False,False,True,False,True,False,True,False,True,False
141023,56.0,240.737339,5.987920,228.186356,False,False,False,False,True,False,...,True,False,True,False,True,False,True,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209741,53.0,259.851300,0.196719,215.601872,True,False,False,False,False,False,...,True,False,True,False,False,True,True,False,False,True
22879,34.0,1146.101785,-4.687871,1268.089349,False,True,False,False,False,False,...,True,False,True,False,True,False,True,False,False,True
130363,31.0,30.830676,-0.482484,-4.751058,False,False,False,False,True,False,...,False,True,True,False,True,False,True,False,False,True
87099,24.0,792.767525,6.663159,676.988885,False,True,False,False,False,False,...,True,False,False,True,True,False,True,False,False,True


In [23]:
# Make predictions with your model using the filtered new data
predictions = pipeline_RF.predict_proba(X_new_filtered)[:, 1]

# Add the predictions as a new column in the DataFrame
X_new_pred = X_new_filtered.copy()
X_new_pred.loc[:, 'predictions'] = predictions
X_new_pred

,age,balance,average_spending,average_balance,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,...,education_tertiary,has_deposits_no,has_deposits_yes,loan_no,loan_yes,has_insurance_n,has_insurance_y,has_mortgage_no,has_mortgage_yes,predictions
265901,41.0,175.445269,-7.454485,203.907928,False,False,False,False,True,False,...,True,True,False,True,False,True,False,True,False,0.286743
175317,34.0,2559.041192,0.444639,2544.965438,False,False,False,False,True,False,...,True,True,False,True,False,True,False,False,True,0.155374
191645,34.0,94.055761,2.436701,-15.679495,False,False,False,False,True,False,...,True,False,True,True,False,True,False,True,False,0.585249
55873,75.0,37136.144031,1.532749,37131.526158,False,False,False,False,False,True,...,False,True,False,True,False,True,False,True,False,0.505362
141023,56.0,240.737339,5.987920,228.186356,False,False,False,False,True,False,...,False,True,False,True,False,True,False,False,True,0.139255
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209741,53.0,259.851300,0.196719,215.601872,True,False,False,False,False,False,...,False,True,False,False,True,True,False,False,True,0.157856
22879,34.0,1146.101785,-4.687871,1268.089349,False,True,False,False,False,False,...,False,True,False,True,False,True,False,False,True,0.093717
130363,31.0,30.830676,-0.482484,-4.751058,False,False,False,False,True,False,...,True,True,False,True,False,True,False,False,True,0.145024
87099,24.0,792.767525,6.663159,676.988885,False,True,False,False,False,False,...,False,False,True,True,False,True,False,False,True,0.485644


In [24]:
# Sort the DataFrame by the 'predictions' column in descending order
X_new_pred = X_new_pred.sort_values('predictions', ascending=False)
X_new_pred

,age,balance,average_spending,average_balance,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,...,education_tertiary,has_deposits_no,has_deposits_yes,loan_no,loan_yes,has_insurance_n,has_insurance_y,has_mortgage_no,has_mortgage_yes,predictions
2469,79.0,144.187196,5.246050,75.581612,False,False,False,False,False,True,...,False,False,True,True,False,True,False,True,False,0.759048
101945,66.0,114.029128,5.654960,37.216584,False,False,False,False,False,True,...,False,False,True,True,False,True,False,True,False,0.752783
225575,78.0,550.918659,2.935134,552.989126,False,False,False,False,False,True,...,False,False,True,True,False,True,False,True,False,0.750691
196299,69.0,208.267122,3.242535,230.139893,False,False,False,False,False,True,...,True,False,True,True,False,True,False,True,False,0.747949
205529,74.0,29066.902642,2.040186,29065.034642,False,False,False,False,False,True,...,False,False,True,True,False,True,False,True,False,0.747855
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203917,33.0,-339.883514,-3.943638,-300.911123,False,True,False,False,False,False,...,False,True,False,False,True,False,True,False,True,0.032044
249885,32.0,-285.315750,-5.025765,-195.839370,False,True,False,False,False,False,...,False,True,False,False,True,True,False,False,True,0.031657
160731,37.0,-364.069743,2.760169,-398.458759,False,True,False,False,False,False,...,False,True,False,False,True,True,False,False,True,0.031295
64999,40.0,-332.928093,-0.099938,-323.258860,False,True,False,False,False,False,...,False,True,False,False,True,True,False,False,True,0.030618


In [25]:
# Select only the top 3000 success_probablity score clients
top_3000_target = X_new_pred.head(3000)
top_3000_target

,age,balance,average_spending,average_balance,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,...,education_tertiary,has_deposits_no,has_deposits_yes,loan_no,loan_yes,has_insurance_n,has_insurance_y,has_mortgage_no,has_mortgage_yes,predictions
2469,79.0,144.187196,5.246050,75.581612,False,False,False,False,False,True,...,False,False,True,True,False,True,False,True,False,0.759048
101945,66.0,114.029128,5.654960,37.216584,False,False,False,False,False,True,...,False,False,True,True,False,True,False,True,False,0.752783
225575,78.0,550.918659,2.935134,552.989126,False,False,False,False,False,True,...,False,False,True,True,False,True,False,True,False,0.750691
196299,69.0,208.267122,3.242535,230.139893,False,False,False,False,False,True,...,True,False,True,True,False,True,False,True,False,0.747949
205529,74.0,29066.902642,2.040186,29065.034642,False,False,False,False,False,True,...,False,False,True,True,False,True,False,True,False,0.747855
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160679,34.0,156.207742,0.862121,83.603689,False,False,False,False,False,False,...,False,False,True,True,False,True,False,False,True,0.475047
51219,29.0,289.546048,-1.684720,268.398573,False,True,False,False,False,False,...,False,False,True,True,False,True,False,False,True,0.475024
232257,34.0,2693.385596,-6.943313,2729.267295,False,False,False,False,False,False,...,False,False,True,True,False,True,False,False,True,0.474805
80495,33.0,199.894151,1.541931,200.418870,False,False,False,False,False,False,...,False,False,True,True,False,True,False,False,True,0.474805


In [26]:
# Save the 'client_id's to a CSV file
top_3000_target.to_csv('top_3000_potencial_client_RF__.csv', index=False)